# The GCR Algorithm
An iterative Krylov subspace numeric meth(od) used for solving systems of linear equations.

### Importing some motherfucking libraries

In [25]:
using LinearAlgebra
using Printf

### Function that computes the GCR

Input parameters:
- `A`: $N \times N$ square matrix of the system $Ax = b$
- `b`: $N$-length column vector that is the right hand side of the linear system
- `tolerance`: the amount of $\Delta$ in the residual left before declaring convergence
- `max_iter`: the total number of iterations to perform the GCR algorithm before exiting
- `pp`: flag variable that prints the residual norm and scaled residual norm for each iteration of $k$; set to 1 to print to console and any other integer for no console output

In [26]:
function GCR(A, b, tolerance, max_iter, pp)
    x = zeros(length(b))
    residual = copy(b);
    res_norm_init = norm(residual);
    
    res_norms = [res_norm_init];
    k = 0;
    p = Float64[];
    Ap = Float64[];
    while (res_norms[k+1]/res_norms[1] > tolerance) && (k <= max_iter)
        k = k+1
        if pp == 1
            @printf("\n----------------- ITERATION %d -----------------\n", k)
            @printf("\nResidual norm: %lf", res_norms[k]);
            @printf("\nScaled res norm %lf\n", res_norms[k]/res_norms[1]);
        end
            
        # Use the residual as the first guess for the ne search direction and computer its image
        if k == 1
            p = copy(residual);
            Ap = A * p[:,k];
            
        elseif k > 1
            p = cat(p, residual, dims=(2,2))
            Ap = cat(Ap, A * p[:,k], dims=(2,2))
        end
        
        # Make the new Ap vector orthogonal to the previous Ap vectors, and the p vectors A^TA orthogonal to the previous p vectors
        # noticed that if you know A is symmetric you can save computation by limiting the for loop to just j=k-1 
        
        if k > 1
            for j in 1:k-1
               beta    = Ap[:,k]' * Ap[:,j];
               p[:,k]  =  p[:,k] - beta *  p[:,j];
               Ap[:,k] = Ap[:,k] - beta * Ap[:,j];
                
            end
        end
        # Make the orthogonal Ap vector of unit length, and scale the p vector so that A * p  is of unit length
        norm_Ap = norm(Ap[:,k]);
        Ap[:,k] = Ap[:,k]/norm_Ap;
        p[:,k] =  p[:,k]/norm_Ap;
        # Determine the optimal amount to change x in the p direction by projecting r onto Ap
        alpha = residual' * Ap[:,k];
        # Update x and r
        x = x + alpha *  p[:,k];
        residual = residual - alpha * Ap[:,k];

        # Save the norm of r
        res_norms = cat(res_norms, norm(residual), dims=(2,2));
    end

    # Tell bitches whether or not this shit converged
    if res_norms[k+1]> (tolerance * res_norms[1])
        print("\n****************************************************************************************\n")
        @printf("* Boohoo, the GCR algorithm did NOT converge! Maximum number of iterations reached >:( *");
        print("\n****************************************************************************************\n")
        x = [];
    else
        print("\n******************************************************************\n")
        @printf("*   SLAY BESTIE! The GCR algorithm converged in %d iterations!   *", k);
        print("\n******************************************************************\n")
    end

    # % Scale the r_norms with respect to the initial residual norm
    res_norms = res_norms / res_norms[1];
end

GCR (generic function with 2 methods)

# Creating test data and calling function
Originally taken from `MATLAB`, given are some equivalent test commands to compare the results between `MATTY` and `Julia`. Enter the commented command into the command line of `MATTY BOI` in the same directory as your `tgcr.m` file to compare results.

In [27]:
# equivalent matlab test commands
# A = [420 0 0 0 0; 0 69 0 0 0; 0 0 420 0 0; 0 0 0 69 0; 0 0 0 0 22]; b = [4; 3; 2; 1; 5]; tgcr(A, b, 0.0001, 300);
# A = [420 2 0 0 0; 0 33 0 0 0; 0 0 62 0 0; 0 0 0 14 0; 0 0 0 0 22]; b = [4; 3; 2; 1; 5]; tgcr(A, b, 0.0001, 300);

#creating test data
N = 5;
A = float([420 0 0 0 0; 0 69 0 0 0; 0 0 420 0 0; 0 0 0 69 0; 0 0 0 0 22]);
b = float([4; 3; 2; 1; 5]);
tol = 0.0001;
iter = 300;
pp = 1; #set to 1 for printing to console, else any int for no printing

In [28]:
GCR(A, b, tol, iter, pp)


----------------- ITERATION 1 -----------------

Residual norm: 7.416198
Scaled res norm 1.000000

----------------- ITERATION 2 -----------------

Residual norm: 5.394237
Scaled res norm 0.727359

----------------- ITERATION 3 -----------------

Residual norm: 2.801851
Scaled res norm 0.377802

******************************************************************
*   SLAY BESTIE! The GCR algorithm converged in 3 iterations!   *
******************************************************************


1×4 Matrix{Float64}:
 1.0  0.727359  0.377802  1.96124e-16